# Install and Import Dependencies

In [1]:
%pip install mediapipe opencv-python
%pip install ultralytics supervision

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
from ultralytics import YOLO
import supervision as sv

In [ ]:
# import uuid
# import os
# mp_hands = mp.solutions.hands

# Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [ ]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

<img src="https://i.imgur.com/qpRACer.png" style="height:300px" >

In [ ]:
# for ldmrk in mp_hands.HandLandmark:
#     print(ldmrk)

# Calculate Angles

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

# Pose Detection

In [4]:
def display_hand(image, hand_pts):
    # DISPLAY AREA OF RIGHT HAND ---------------
    # Calculate the center of the circle in 3D space (x, y, z)
    center_3d = np.mean(hand_pts, axis=0)
    # Calculate the radius of the circle in 3D space based on the average distance from the center to each point
    distances = [np.linalg.norm([p[0] - center_3d[0], p[1] - center_3d[1], p[2] - center_3d[2]]) for p in hand_pts]

    scaling_factor = 3  # scaling factor must be int
    radius_3d = scaling_factor * int(sum(distances ) / len(distances))

    # Draw the circle in 3D space
    cv2.circle(image, (int(center_3d[0]), int(center_3d[1])), radius_3d, (245, 117, 66), thickness=-1)  # -1 thickness for a filled circle

In [5]:
def feet_pts(ankle, heel, index, frame_shape_1, frame_shape_0):
    return np.array([
        [int(ankle.x * frame_shape_1), int(ankle.y * frame_shape_0)],
        [int(heel.x * frame_shape_1), int(heel.y * frame_shape_0)],
        [int(index.x * frame_shape_1), int(index.y * frame_shape_0)]
        ], np.int32)

In [6]:
def hand_pts(pinky, index, thumb, wrist, frame_shape_1, frame_shape_0):
    return np.array([
        [int(pinky.x * frame_shape_1), int(pinky.y * frame_shape_0)],
        [int(index.x * frame_shape_1), int(index.y * frame_shape_0)],
        [int(thumb.x * frame_shape_1), int(thumb.y * frame_shape_0)],
        [int(wrist.x * frame_shape_1), int(wrist.y * frame_shape_0)]
        ], np.int32)

In [7]:
def display_coords(d):
    max_key_length = max(len(key) for key in d.keys())
    max_x_length = max(len(str(value.x)) for value in d.values())
    max_y_length = max(len(str(value.y)) for value in d.values())
    max_z_length = max(len(str(value.z)) for value in d.values())

    for point, coords in d.items():
        formatted_x = str(coords.x).rjust(max_x_length)
        formatted_y = str(coords.y).rjust(max_y_length)
        formatted_z = str(coords.z).rjust(max_z_length)
        print(f"{point.ljust(max_key_length)}: x = {formatted_x}, y = {formatted_y}, z = {formatted_z}")
    print("\n")

In [8]:
def is_within_hold(limb, detection):
    x, y = limb.x, limb.y
    x1, y1, x2, y2 = detection[0], detection[1], detection[2], detection[3]
    # Check if limb coordinates are within the bounding box
    return x1 <= x <= x2 and y1 <= y <= y2

In [9]:
def get_center_point(d, limb, right_foot_pts, left_foot_pts, right_hand_pts, left_hand_pts):
    r_foot = ["right_ankle", "right_heel", "right_foot_index"]
    l_foot = ["left_ankle", "left_heel", "left_foot_index"]
    r_hand = ["right_pinky", "right_index","right_thumb", "right_wrist"]
    l_hand = ["left_pinky", "left_index","left_thumb", "left_wrist"]
    if limb in r_foot:
        return np.mean(right_foot_pts, axis=0)
    elif limb in l_foot:
        return np.mean(left_foot_pts, axis=0)
    elif limb in r_hand:
        return np.mean(right_hand_pts, axis=0)
    elif limb in l_hand:
        return np.mean(left_hand_pts, axis=0)
    else:
        return np.array([d[limb].x, d[limb].y], np.int32)

In [10]:
# TODO: function that checks what holds the person is on
# A hold corresponding to right hand, left hand, right foot, left foot
def get_curr_position(d, detections):
    extremities = ["right_foot", "left_foot", "right_hand","left_hand"]
    for limb, coords in d.items():
        for detection in detections:
            # yields opposite corners
            x1, y1, x2, y2 = detection[0], detection[1], detection[2], detection[3]
            limb_x, limb_y = coords.x, coords.y

            if (limb in extremities) and x1 <= limb_x <= x2 and y1 <= limb_y <= y2: # Within bounds
                # save the coordinates
                pass


In [11]:
def get_relative_position(center_limb_pt, rock_hold):
    # points of rock_hold
    x1, y1, x2, y2 = rock_hold[0][0], rock_hold[0][1], rock_hold[0][2], rock_hold[0][3]
    # print("Rock_coords:", x1, y1, x2, y2)
    mean_rock_coord = np.mean(np.array([[x1, y1], [x2, y2]]), axis=0)
    # print("M:", mean_rock_coord)
    # print("C:", center_limb_pt[:2])
    return np.linalg.norm(abs(center_limb_pt[:2] - mean_rock_coord))

In [12]:
# JUST THE POSE

cap = cv2.VideoCapture(0)

model = YOLO('bestHuge.pt')
box_annotator = sv.BoxAnnotator(thickness=2, text_thickness=2, text_scale=1)

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)


        # YOLO results
        results2 = model(frame, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results2)
        detections = detections[detections.confidence > 0.8]

        frame = box_annotator.annotate(scene=image, detections=detections)


        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        #                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
        #                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            pose_landmark = mp_pose.PoseLandmark

            d = {}  # body dictionary

            # Upper body coordinates
            d["left_shoulder"] = landmarks[pose_landmark.LEFT_SHOULDER.value]
            d["right_shoulder"] = landmarks[pose_landmark.RIGHT_SHOULDER]

            d["left_elbow"] = landmarks[pose_landmark.LEFT_ELBOW.value]
            d["right_elbow"] = landmarks[pose_landmark.RIGHT_ELBOW.value]

            frame_shape_0, frame_shape_1 = frame.shape[0], frame.shape[1]

            # LEFT HAND
            d["left_pinky"] = landmarks[pose_landmark.LEFT_PINKY.value]
            d["left_index"] = landmarks[pose_landmark.LEFT_INDEX.value]
            d["left_thumb"] = landmarks[pose_landmark.LEFT_THUMB.value]
            d["left_wrist"] = landmarks[pose_landmark.LEFT_WRIST.value]
            left_hand_pts = hand_pts(d["left_pinky"], d["left_index"], d["left_thumb"], d["left_wrist"], frame_shape_1, frame_shape_0)

            # RIGHT HAND
            d["right_pinky"] = landmarks[pose_landmark.RIGHT_PINKY.value]
            d["right_index"] = landmarks[pose_landmark.RIGHT_INDEX.value]
            d["right_thumb"] = landmarks[pose_landmark.RIGHT_THUMB.value]
            d["right_wrist"] = landmarks[pose_landmark.RIGHT_WRIST.value]
            right_hand_pts = hand_pts(d["right_pinky"], d["right_index"], d["right_thumb"], d["right_wrist"], frame_shape_1, frame_shape_0)

            # Lower body coordinates
            d["left_knee"] = landmarks[pose_landmark.LEFT_KNEE.value]
            d["right_knee"] = landmarks[pose_landmark.RIGHT_KNEE.value]

            # LEFT FOOT
            d["left_ankle"] = landmarks[pose_landmark.LEFT_ANKLE.value]
            d["left_heel"] = landmarks[pose_landmark.LEFT_HEEL.value]
            d["left_foot_index"] = landmarks[pose_landmark.LEFT_FOOT_INDEX.value]
            left_foot_pts = feet_pts(d["left_ankle"], d["left_heel"], d["left_foot_index"], frame_shape_1, frame_shape_0)

            # RIGHT FOOT
            d["right_ankle"] = landmarks[pose_landmark.RIGHT_ANKLE.value]
            d["right_heel"] = landmarks[pose_landmark.RIGHT_HEEL.value]
            d["right_foot_index"] = landmarks[pose_landmark.RIGHT_FOOT_INDEX.value]
            right_foot_pts = feet_pts(d["right_ankle"], d["right_heel"], d["right_foot_index"], frame_shape_1, frame_shape_0)

            # Display Coordinates
            # display_coords(d)

            for detection in detections:
                # currently only for right_hand
                point = get_center_point(d, "right_thumb", right_foot_pts, left_foot_pts, right_hand_pts, left_hand_pts)
                get_relative_position(point, detection)
                print(f"Relative position: {get_relative_position(point, detection)} " + " " * 20, end='\r')

            # center_2d = np.mean(right_hand_pts, axis=0)[:2]

            cv2.fillPoly(image, [right_foot_pts], (245, 117, 66))
            cv2.fillPoly(image, [left_foot_pts], (245, 117, 66))

            display_hand(image,right_hand_pts)
            display_hand(image,left_hand_pts)

            # Calculate angle
            # angle = calculate_angle(shoulder, elbow, wrist)
            # Visualize angle
            # cv2.putText(image, str(angle),
            #                tuple(np.multiply(elbow, [640, 480]).astype(int)),
            #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                     )

        except:
            pass

        # print(results.pose_landmarks)
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        #                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
        #                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        cv2.imshow('Pose Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


KeyboardInterrupt: 

In [ ]:
# JUST THE HANDS
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()

        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Detections
        print(results)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
            
        # Save our image    
        cv2.imwrite(os.path.join('Output Images', '{}.jpg'.format(uuid.uuid1())), image)
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# THE HANDS AND THE POSE
# TODO: REMOVE THE mini hands from the pose
cap = cv2.VideoCapture(0)
# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose, mp_hands.Hands(min_detection_confidence=0.3, min_tracking_confidence=0.3) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        # Get image used for pose estimation
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)
        hand_results = hands.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            hand_landmarks = hand_results.multi_hand_landmarks

            # PRINTING THE POSE COORDINATES
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            print("SHOULDERS")
            print("Left_Shoulder --- x: " + str(left_shoulder.x) + "y: " + str(left_shoulder.y) +  "z: " + str(left_shoulder.z))
            print("Right_Shoulder --- x: " + str(right_shoulder.x) + "y: " + str(right_shoulder.y) + "z: " + str(right_shoulder.z))

            print("ELBOWS")
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            print("Left_elbow --- x: " + str(left_elbow.x) + "y: " + str(left_elbow.y) + "z: " + str(left_elbow.z))
            print("Right_elbow --- x: " + str(right_elbow.x) + "y: " + str(right_elbow.y) + "z: " + str(right_elbow.z))

            # LEFT HAND
            left_pinky = landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value]
            left_index = landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value]
            left_thumb = landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            print("LEFT HAND")
            print("Left_pinky --- x: " + str(left_pinky.x) + "y: " + str(left_pinky.y) + "z: " + str(left_pinky.z))
            print("left_index --- x: " + str(left_index.x) + "y: " + str(left_index.y) + "z: " + str(left_index.z))
            print("left_thumb --- x: " + str(left_thumb.x) + "y: " + str(left_thumb.y) + "z: " + str(left_thumb.z))
            print("Left_wrist --- x: " + str(left_wrist.x) + "y: " + str(left_wrist.y) + "z: " + str(left_wrist.z))

            # RIGHT HAND
            right_pinky = landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value]
            right_index = landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value]
            right_thumb = landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            print("RIGHT HAND")
            print("Right_pinky --- x: " + str(right_pinky.x) + "y: " + str(right_pinky.y) + "z: " + str(right_pinky.z))
            print("Right_index --- x: " + str(right_index.x) + "y: " + str(right_index.y) + "z: " + str(right_index.z))
            print("Right_thumb --- x: " + str(right_thumb.x) + "y: " + str(right_thumb.y) + "z: " + str(right_thumb.z))
            print("Right_wrist: x: " + str(right_wrist.x) + "y: " + str(right_wrist.y) + "z: " + str(right_wrist.z))

            # Lower body coordinates
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
            print("LEFT KNEE --- x: " + str(left_knee.x) + " y: " + str(left_knee.y) + "z: " + str(left_knee.z))

            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
            print("RIGHT KNEE --- x: " + str(right_knee.x) + " y: " + str(right_knee.y) + "z: " + str(right_knee.z))

            # LEFT FOOT
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
            left_heel = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value]
            left_foot_index = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]
            print("LEFT FOOT")
            print("Left_ankle --- x: " + str(left_ankle.x) + " y: " + str(left_ankle.y) + " z: " + str(left_ankle.z))
            print("Left_heel --- x: " + str(left_heel.x) + " y: " + str(left_heel.y) + " z: " + str(left_heel.z))
            print("Left_foot_index --- x: " + str(left_foot_index.x) + " y: " + str(left_foot_index.y) + " z: " + str(left_foot_index.z))

            # RIGHT FOOT
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
            right_heel = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value]
            right_foot_index = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
            print("RIGHT FOOT")
            print("Right_ankle --- x: " + str(right_ankle.x) + " y: " + str(right_ankle.y) + " z: " + str(right_ankle.z))
            print("Right_heel --- x: " + str(right_heel.x) + " y: " + str(right_heel.y) + " z: " + str(right_heel.z))
            print("Right_foot_index --- x: " + str(right_foot_index.x) + " y: " + str(right_foot_index.y) + " z: " + str(right_foot_index.z))

            frame_shape_0, frame_shape_1 = frame.shape[0], frame.shape[1]

            # DISPLAY AREA OF RIGHT FOOT ---------------
            right_foot_pts = np.array([
                [int(right_ankle.x * frame_shape_1), int(right_ankle.y * frame_shape_0)],
                [int(right_heel.x * frame_shape_1), int(right_heel.y * frame_shape_0)],
                [int(right_foot_index.x * frame_shape_1), int(right_foot_index.y * frame_shape_0)]
            ], np.int32).reshape((-1, 1, 2))

            # Fill the area of the right foot with a specific color
            cv2.fillPoly(image, [right_foot_pts], (245, 117, 66))

            # DISPLAY AREA OF LEFT FOOT ---------------
            left_foot_pts = np.array([
                [int(left_ankle.x * frame_shape_1), int(left_ankle.y * frame_shape_0)],
                [int(left_heel.x * frame_shape_1), int(left_heel.y * frame_shape_0)],
                [int(left_foot_index.x * frame_shape_1), int(left_foot_index.y * frame_shape_0)]
            ], np.int32).reshape((-1, 1, 2))

            # Fill the area of the left foot with a specific color
            cv2.fillPoly(image, [left_foot_pts], (245, 117, 66))
    
            # DISPLAY AREA OF RIGHT HAND ---------------
            right_hand_pts = np.array([
                [int(right_pinky.x * frame_shape_1), int(right_pinky.y * frame_shape_0), right_pinky.z],
                [int(right_index.x * frame_shape_1), int(right_index.y * frame_shape_0), right_index.z],
                [int(right_thumb.x * frame_shape_1), int(right_thumb.y * frame_shape_0), right_thumb.z],
                [int(right_wrist.x * frame_shape_1), int(right_wrist.y * frame_shape_0), right_wrist.z]
            ], np.int32)

            # Calculate the center of the circle in 3D space (x, y, z)
            right_center_3d = np.mean(right_hand_pts, axis=0)

            # Calculate the radius of the circle in 3D space based on the average distance from the center to each point
            right_distances = [np.linalg.norm([p[0] - right_center_3d[0], p[1] - right_center_3d[1], p[2] - right_center_3d[2]]) for p in right_hand_pts]
            radius_3d = int(sum(right_distances ) / len(right_distances))

            # Draw the circle in 3D space
            cv2.circle(image, (int(right_center_3d[0]), int(right_center_3d[1])), radius_3d, (245, 117, 66), thickness=-1)  # -1 thickness for a filled circle

            # DISPLAY AREA OF LEFT HAND ---------------
            left_hand_pts = np.array([
                [int(left_pinky.x * frame_shape_1), int(left_pinky.y * frame_shape_0), left_pinky.z],
                [int(left_index.x * frame_shape_1), int(left_index.y * frame_shape_0), left_index.z],
                [int(left_thumb.x * frame_shape_1), int(left_thumb.y * frame_shape_0), left_thumb.z],
                [int(left_wrist.x * frame_shape_1), int(left_wrist.y * frame_shape_0), left_wrist.z]
            ], np.int32)

            # Calculate the center of the circle in 3D space (x, y, z)
            left_center_3d = np.mean(left_hand_pts, axis=0)

            # Calculate the radius of the circle in 3D space based on the average distance from the center to each point
            left_distances = [np.linalg.norm([p[0] - left_center_3d[0], p[1] - left_center_3d[1], p[2] - left_center_3d[2]]) for p in left_hand_pts]
            radius_3d = int(sum(left_distances ) / len(left_distances))

            # Draw the circle in 3D space
            cv2.circle(image, (int(left_center_3d[0]), int(left_center_3d[1])), radius_3d, (245, 117, 66), thickness=-1)  # -1 thickness for a filled circle

            # Calculate angle
            # angle = calculate_angle(shoulder, elbow, wrist)
            # Visualize angle
            # cv2.putText(image, str(angle),
            #                tuple(np.multiply(elbow, [640, 480]).astype(int)),
            #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                     )

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
            
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),)

        except:
            pass

        # print(results.pose_landmarks)
        # Render detections

        cv2.imshow('Pose Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()